In [1]:
from transformers import (

    AutoFeatureExtractor, 

    AutoTokenizer, 

    VisionEncoderDecoderModel,

    Seq2SeqTrainingArguments,

    Seq2SeqTrainer, 

    default_data_collator)

!pip install datasets

from torch.utils.data import Dataset



import pandas as pd

from sklearn.model_selection import train_test_split



from pathlib import Path

from PIL import Image

from datasets import load_dataset

import torch

import pandas as np

import matplotlib.pyplot as plt

import random

import tqdm

import nltk

from torch.utils.data import DataLoader

from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction



# remove warnings

import warnings

warnings.filterwarnings('ignore')

In [3]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer

import torch

from PIL import Image


model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.47.0"
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "activation_function": "gelu_new",
  "add_cross_attention": true,
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "decoder_start_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_rang

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(i

In [ ]:
from datasets import load_dataset

# Example: Loading dataset from Hugging Face Hub (adjust as needed)
ds = load_dataset("", split="train")  # Replace with your dataset and split

# Split the dataset into train and temp (for validation and test)
train_val_split = 0.80
train_ds, temp_ds = ds.train_test_split(test_size=1 - train_val_split).values()

# Split the temp dataset into validation and test sets
val_size = 0.5
val_ds, test_ds = temp_ds.train_test_split(test_size=val_size).values()

README.md:   0%|          | 0.00/357 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/251M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/251M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19455 [00:00<?, ? examples/s]

In [3]:
max_length = 512

num_beams = 4

gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

In [6]:
def predict_step(dataset, indices):
    images = []
    actual_captions = []  # List to store actual captions

    for idx in indices:
        i_image = dataset[idx]['image']
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")
        images.append(i_image)
        actual_captions.append(dataset[idx]['findings'])  # Add actual caption to the list

    # Extract pixel values and attention masks
    inputs = feature_extractor(images=images, return_tensors="pt")
    pixel_values = inputs.pixel_values.to(device)
    attention_mask = inputs.attention_mask.to(device) if 'attention_mask' in inputs else None

    gen_kwargs = {"max_length": 512, "num_beams": 4}  # Adjust parameters as needed

    # Include attention_mask in generate call if available
    if attention_mask is not None:
        output_ids = model.generate(pixel_values, attention_mask=attention_mask, **gen_kwargs)
    else:
        output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]

    return preds, actual_captions  # Return both predictions and actual captions

# Example usage: predict captions for the first two images in the dataset
predictions, actual_captions = predict_step(ds, [0, 1])

for pred, actual in zip(predictions, actual_captions):
    print(f"Generated Findings: {pred}")
    print(f"Actual Findings: {actual}")
    print("-----")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Generated Findings: a black and white photo of a person holding a black and white vase
Actual Findings: The lungs are clear of focal consolidation, pleural effusion or pneumothorax. The heart size is normal. The mediastinal contours are normal. Multiple surgical clips project over the left breast, and old left rib fractures are noted.
-----
Generated Findings: a black and white photo of a black and white photo
Actual Findings: Lung volumes remain low. There are innumerable bilateral scattered small pulmonary nodules which are better demonstrated on recent CT. Mild pulmonary vascular congestion is stable. The cardiomediastinal silhouette and hilar contours are unchanged. Small pleural effusion in the right middle fissure is new. There is no new focal opacity to suggest pneumonia. There is no pneumothorax.
-----


In [7]:
from torch.utils.data import DataLoader

from transformers import default_data_collator



# Define a function to prepare the dataset for training

def preprocess_data(examples):

    # Convert images to RGB and apply feature extractor

    images = [i.convert("RGB") if i.mode != "RGB" else i for i in examples['image']]

    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values

    text_inputs = tokenizer(examples['findings'], padding="max_length", truncation=True, max_length=512)

    text_inputs['labels'] = text_inputs.input_ids

    text_inputs['pixel_values'] = pixel_values



    return text_inputs


# Preprocess the datasets

train_dataset = train_ds.map(preprocess_data, batched=True, remove_columns=['image', 'findings', 'impression'])

val_dataset = val_ds.map(preprocess_data, batched=True, remove_columns=['image', 'findings', 'impression'])

test_dataset = test_ds.map(preprocess_data, batched=True, remove_columns=['image', 'findings', 'impression'])



# Print dataset sizes to check

print(f"Train dataset shape: {len(train_dataset)}")

print(f"Validation dataset shape: {len(val_dataset)}")

Map:   0%|          | 0/15564 [00:00<?, ? examples/s]

Map:   0%|          | 0/1945 [00:00<?, ? examples/s]

Map:   0%|          | 0/1946 [00:00<?, ? examples/s]

Train dataset shape: 15564
Validation dataset shape: 1945


In [9]:
import wandb

In [ ]:
# Initialize Weights & Biases
wandb_api = ""
wandb.login(key=wandb_api)
wandb.init(project="findings_generation",
          name="vit-findings-generation")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: itsanmolgupta (anmol_creations). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [11]:
from transformers import Trainer, TrainingArguments



# Training arguments

training_args = TrainingArguments(

    output_dir="./output",

    per_device_train_batch_size=8,
    
    per_device_eval_batch_size=8,

    eval_strategy="epoch",

    num_train_epochs=10,  # Adjust as needed

    save_steps=500,

    save_total_limit=2,

    logging_dir="./logs",

    logging_steps=100,

    learning_rate=5e-5,

)



# Define a data collator

data_collator = default_data_collator



# Create Trainer instance

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=train_dataset,

    data_collator=data_collator,

    eval_dataset=val_dataset,  # Pass the validation dataset here

)



# Fine-tune the model

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,0.217500,0.206469
2,0.203400,0.192712
3,0.192400,0.186247
4,0.179200,0.182160
5,0.172000,0.180814
6,0.161300,0.181140
7,0.157700,0.183833
8,0.139300,0.187700
9,0.141300,0.191353
10,0.128000,0.196180


TrainOutput(global_step=19460, training_loss=0.1725445248485224, metrics={'train_runtime': 31928.8051, 'train_samples_per_second': 4.875, 'train_steps_per_second': 0.609, 'total_flos': 2.808740001393672e+19, 'train_loss': 0.1725445248485224, 'epoch': 10.0})

In [15]:
!pip install evaluate rouge-score -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00


In [16]:
!pip install nltk==3.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 35.0 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434676 sha256=eb712358d979514e65d9abcd91e57da22f1755ab7562cd096066a03a21c0932f
  Stored in directory: /root/.cache/pip/wheels/35/ab/82/f9667f6f884d272670a15382599a9c753a1dfdc83f7412e37d
Successfully built nltk
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.5 which is incompatible.


In [66]:
!pip install sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.5 MB/s eta 0:00:00


In [11]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer

# Ensure you have the necessary NLTK data files
# nltk.download('punkt')

def predict_step(dataset, indices):
    images = []
    actual_captions = []  # List to store actual captions
    for idx in indices:
        i_image = dataset[idx]['image']
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")
        images.append(i_image)
        actual_captions.append(dataset[idx]['findings'])  # Add actual caption to the list

    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    gen_kwargs = {"max_length": 512, "num_beams": 4}  # Adjust parameters as needed
    output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]

    return preds, actual_captions  # Return both predictions and actual captions

# Example usage: predict captions for the first two images in the dataset
predictions, actual_captions = predict_step(train_ds, [100])

# Evaluate BLEU score
bleu_scores = []

for pred, actual in zip(predictions, actual_captions):
    # Tokenize the sentences
    reference = [word_tokenize(actual.lower())]  # Reference captions (list of lists)
    candidate = word_tokenize(pred.lower())       # Generated captions (list)
    
    # Calculate BLEU score
    score = sentence_bleu(reference, candidate)
    bleu_scores.append(score)

# Evaluate ROUGE score
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
for pred, actual in zip(predictions, actual_captions):
    scores = rouge.score(actual, pred)
    for key in rouge_scores:
        rouge_scores[key].append(scores[key].fmeasure)

# Print predictions, actual captions, BLEU scores, and ROUGE scores
for idx, (pred, actual, bleu_score) in enumerate(zip(predictions, actual_captions, bleu_scores)):
    print(f"Actual Findings: \n{actual}")
    print("-----")
    print(f"Generated Findings: \n{pred}")
    print("-----")
    print(f"BLEU score: {bleu_score:.4f}")
    for rouge_key in rouge_scores:
        print(f"{rouge_key.upper()} score: {rouge_scores[rouge_key][idx]:.4f}")

Actual Findings: 
AP portable upright view of the chest. Left chest wall AICD is again seen with leads extending to the region of the right atrium and right ventricle. The heart remains markedly enlarged. There is mild elevation of the right hemidiaphragm again noted. At least mild pulmonary edema is noted. No large effusion or pneumothorax. Mediastinal contour is unchanged. Bony structures are intact peer
-----
Generated Findings: 
AP portable upright view of the chest. Left chest wall AICD is again seen with leads extending to the region the right atrium and right ventricle. Cardiomegaly is again noted with mild pulmonary edema. No large effusion or pneumothorax. Bony structures are intact.
-----
BLEU score: 0.4881
ROUGE1 score: 0.7736
ROUGE2 score: 0.6538
ROUGEL score: 0.7736


In [13]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer

# Ensure you have the necessary NLTK data files
# nltk.download('punkt')

def predict_step(dataset, indices):
    images = []
    actual_captions = []  # List to store actual captions
    for idx in indices:
        i_image = dataset[idx]['image']
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")
        images.append(i_image)
        actual_captions.append(dataset[idx]['findings'])  # Add actual caption to the list

    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    gen_kwargs = {"max_length": 512, "num_beams": 4}  # Adjust parameters as needed
    output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]

    return preds, actual_captions  # Return both predictions and actual captions

# Example usage: predict captions for the first two images in the dataset
predictions, actual_captions = predict_step(train_ds, [500])

# Evaluate BLEU score
bleu_scores = []

for pred, actual in zip(predictions, actual_captions):
    # Tokenize the sentences
    reference = [word_tokenize(actual.lower())]  # Reference captions (list of lists)
    candidate = word_tokenize(pred.lower())       # Generated captions (list)
    
    # Calculate BLEU score
    score = sentence_bleu(reference, candidate)
    bleu_scores.append(score)

# Evaluate ROUGE score
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
for pred, actual in zip(predictions, actual_captions):
    scores = rouge.score(actual, pred)
    for key in rouge_scores:
        rouge_scores[key].append(scores[key].fmeasure)

# Print predictions, actual captions, BLEU scores, and ROUGE scores
for idx, (pred, actual, bleu_score) in enumerate(zip(predictions, actual_captions, bleu_scores)):
    print(f"Actual Findings: \n{actual}")
    print("-----")
    print(f"Generated Findings: \n{pred}")
    print("-----")
    print(f"BLEU score: {bleu_score:.4f}")
    for rouge_key in rouge_scores:
        print(f"{rouge_key.upper()} score: {rouge_scores[rouge_key][idx]:.4f}")

Actual Findings: 
Portable single frontal chest radiograph was obtained with the patient in semi-upright position. There has been interval increase in the opacity projecting over the left hemithorax. There is complete opacification of the left lung base with air bronchograms and obscuration of the left hemidiaphragm. There has also been interval increase in the right base opacity. There is no pneumothorax. The heart size is difficult to assess given parenchymal abnormalities. 
-----
Generated Findings: 
There has been interval placement of a left internal jugular central venous catheter with tip terminating in the mid SVC. The tip of the endotracheal tube projects over the mid thoracic trachea. The tip of the enteric tube projects below the level of the diaphragm but beyond the field of view of this radiograph. There is persistent opacification of the left lung base with opacification of the right lung base likely reflecting a combination of pleural effusion and atelectasis. There is n

In [6]:
!pip install sacrebleu rouge_score -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.9 MB/s eta 0:00:00


In [9]:
from sacrebleu.metrics import BLEU
from rouge_score import rouge_scorer
from torch.utils.data import DataLoader

# Initialize BLEU scorer
bleu = BLEU()

# Initialize ROUGE scorer
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Custom collate function for DataLoader
def custom_collate_fn(batch):
    images = []
    findings = []
    for item in batch:
        # Convert image to RGB
        img = item['image']
        img = img.convert("RGB") if img.mode != "RGB" else img
        images.append(img)
        findings.append(item['findings'])
    return {'images': images, 'findings': findings}

def evaluate_test_set(dataset, batch_size=8):
    """
    Evaluate BLEU and ROUGE scores on the test set with minimal RAM usage.

    Args:
        dataset: The test dataset.
        batch_size: Number of samples to process in each batch.
    
    Returns:
        None. Prints average BLEU and ROUGE scores.
    """
    # Create DataLoader for batch processing with custom collate function
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

    bleu_scores = []
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for batch in dataloader:
        # Extract images and findings
        images = batch['images']
        actuals = batch['findings']

        # Predict findings for the batch
        inputs = feature_extractor(images=images, return_tensors="pt").to(device)
        pixel_values = inputs.pixel_values
        output_ids = model.generate(pixel_values, max_length=512, num_beams=4)
        predictions = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        predictions = [pred.strip() for pred in predictions]

        # Compute scores for the batch
        for pred, actual in zip(predictions, actuals):
            # BLEU score
            bleu_score = bleu.sentence_score(pred, [actual]).score
            bleu_scores.append(bleu_score)

            # ROUGE scores
            rouge_result = rouge.score(pred, actual)
            for key in rouge_scores.keys():
                rouge_scores[key].append(rouge_result[key].fmeasure)

    # Print average scores
    print("Train Set Evaluation:")
    print(f"Average BLEU: {sum(bleu_scores) / len(bleu_scores):.4f}")
    for key in rouge_scores.keys():
        print(f"Average {key.upper()}: {sum(rouge_scores[key]) / len(rouge_scores[key]):.4f}")
    print("-----")

# Call the function to evaluate the test set
evaluate_test_set(train_ds)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Train Set Evaluation:
Average BLEU: 13.8785
Average ROUGE1: 0.4338
Average ROUGE2: 0.1990
Average ROUGEL: 0.3156
-----


In [7]:
from sacrebleu.metrics import BLEU
from rouge_score import rouge_scorer
from torch.utils.data import DataLoader

# Initialize BLEU scorer
bleu = BLEU()

# Initialize ROUGE scorer
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Custom collate function for DataLoader
def custom_collate_fn(batch):
    images = []
    findings = []
    for item in batch:
        # Convert image to RGB
        img = item['image']
        img = img.convert("RGB") if img.mode != "RGB" else img
        images.append(img)
        findings.append(item['findings'])
    return {'images': images, 'findings': findings}

def evaluate_test_set(dataset, batch_size=8):
    """
    Evaluate BLEU and ROUGE scores on the test set with minimal RAM usage.

    Args:
        dataset: The test dataset.
        batch_size: Number of samples to process in each batch.
    
    Returns:
        None. Prints average BLEU and ROUGE scores.
    """
    # Create DataLoader for batch processing with custom collate function
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

    bleu_scores = []
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for batch in dataloader:
        # Extract images and findings
        images = batch['images']
        actuals = batch['findings']

        # Predict findings for the batch
        inputs = feature_extractor(images=images, return_tensors="pt").to(device)
        pixel_values = inputs.pixel_values
        output_ids = model.generate(pixel_values, max_length=512, num_beams=4)
        predictions = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        predictions = [pred.strip() for pred in predictions]

        # Compute scores for the batch
        for pred, actual in zip(predictions, actuals):
            # BLEU score
            bleu_score = bleu.sentence_score(pred, [actual]).score
            bleu_scores.append(bleu_score)

            # ROUGE scores
            rouge_result = rouge.score(pred, actual)
            for key in rouge_scores.keys():
                rouge_scores[key].append(rouge_result[key].fmeasure)

    # Print average scores
    print("Validation Set Evaluation:")
    print(f"Average BLEU: {sum(bleu_scores) / len(bleu_scores):.4f}")
    for key in rouge_scores.keys():
        print(f"Average {key.upper()}: {sum(rouge_scores[key]) / len(rouge_scores[key]):.4f}")
    print("-----")

# Call the function to evaluate the test set
evaluate_test_set(val_ds)

Validation Set Evaluation:
Average BLEU: 13.3225
Average ROUGE1: 0.4295
Average ROUGE2: 0.1944
Average ROUGEL: 0.3117
-----


In [5]:
from sacrebleu.metrics import BLEU
from rouge_score import rouge_scorer
from torch.utils.data import DataLoader

# Initialize BLEU scorer
bleu = BLEU()

# Initialize ROUGE scorer
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Custom collate function for DataLoader
def custom_collate_fn(batch):
    images = []
    findings = []
    for item in batch:
        # Convert image to RGB
        img = item['image']
        img = img.convert("RGB") if img.mode != "RGB" else img
        images.append(img)
        findings.append(item['findings'])
    return {'images': images, 'findings': findings}

def evaluate_test_set(dataset, batch_size=8):
    """
    Evaluate BLEU and ROUGE scores on the test set with minimal RAM usage.

    Args:
        dataset: The test dataset.
        batch_size: Number of samples to process in each batch.
    
    Returns:
        None. Prints average BLEU and ROUGE scores.
    """
    # Create DataLoader for batch processing with custom collate function
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

    bleu_scores = []
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for batch in dataloader:
        # Extract images and findings
        images = batch['images']
        actuals = batch['findings']

        # Predict findings for the batch
        inputs = feature_extractor(images=images, return_tensors="pt").to(device)
        pixel_values = inputs.pixel_values
        output_ids = model.generate(pixel_values, max_length=512, num_beams=4)
        predictions = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        predictions = [pred.strip() for pred in predictions]

        # Compute scores for the batch
        for pred, actual in zip(predictions, actuals):
            # BLEU score
            bleu_score = bleu.sentence_score(pred, [actual]).score
            bleu_scores.append(bleu_score)

            # ROUGE scores
            rouge_result = rouge.score(pred, actual)
            for key in rouge_scores.keys():
                rouge_scores[key].append(rouge_result[key].fmeasure)

    # Print average scores
    print("Test Set Evaluation:")
    print(f"Average BLEU: {sum(bleu_scores) / len(bleu_scores):.4f}")
    for key in rouge_scores.keys():
        print(f"Average {key.upper()}: {sum(rouge_scores[key]) / len(rouge_scores[key]):.4f}")
    print("-----")

# Call the function to evaluate the test set
evaluate_test_set(test_ds)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Test Set Evaluation:
Average BLEU: 13.4792
Average ROUGE1: 0.4291
Average ROUGE2: 0.1949
Average ROUGEL: 0.3121
-----


In [12]:
# Save the model

model.save_pretrained("./findings-model")

feature_extractor.save_pretrained("./findings-model")

tokenizer.save_pretrained("./findings-model")

('./findings-model/tokenizer_config.json',
 './findings-model/special_tokens_map.json',
 './findings-model/vocab.json',
 './findings-model/merges.txt',
 './findings-model/added_tokens.json',
 './findings-model/tokenizer.json')

In [13]:
import shutil

import os



# Define the directory to zip

model_dir = "./findings-model"

zip_file_path = "./findings-model-vitgpt2.zip"



# Create a zip file of the model directory

shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', model_dir)



# Now, you can use the appropriate method to download the zip file.

# If you're in a Jupyter Notebook, you can use the following code to create a download link:



from IPython.display import FileLink



# Create a download link for the zip file

FileLink(zip_file_path)

/kaggle/working/findings_model-vitgpt2.zip